In [1]:
! conda install -c conda-forge keras==2.3.1 -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.9.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ubuntu/miniconda3/envs/py3

  added / updated specs:
    - keras


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ipython-7.20.0             |   py37h888b3d9_2         1.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.1 MB

The following packages will be UPDATED:

  certifi            pkgs/main::certifi-2020.12.5-py37h06a~ --> conda-forge::certifi-2020.12.5-py37h89c1867_1
  ipython                             7.12.0-py37h5ca1d4c_0 --> 7.20.0-py37h888b3d9_2

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certif

In [2]:

# for keras 2.3.1
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve,auc

import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [18]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
#
# CustomCNN.RMSprop.512_1211.V2.h5

deployed_path = "/mnt/vdb/thesis/CustomCNN.RMSprop.512_1211.V2.h5"
import tensorflow as tf

with tf.device('/cpu:0'):
    learner = load_model(deployed_path)

In [4]:
def toPredictDF(path,to_dir,existing_result):
    appended_pkl = []
    for infile in glob.glob(path):
        #print("Read:",infile)
        file_name = os.path.basename(infile)
        result_path=to_dir+"/"+file_name.replace("pkl", "ML.pkl")
        if result_path in existing_result :
        # print("found then skip : " , result)
            continue
        else:
            df = pd.read_pickle(infile)
            ready_df =df[[ "reps"]]
            df_new = ready_df.reps.apply(pd.Series).astype(np.float64)
            df_new.columns = df_new.columns.astype(str)
            dl = learner.dls.test_dl(df_new)
            _preds,_none ,_y = learner.get_preds(dl=dl, with_decoded=True)
            df.drop(columns=['reps','length'],inplace =True)
            df['class'] = _y
            #print("Save:",result_path)
            df.to_pickle(result_path)
    print("Complete")

def mergeDF(path,to_dir,file_name):
    appended_data = []
    for infile in glob.glob(path):
        #print(infile)
        data = pd.read_pickle(infile)
        # store DataFrame in list
        appended_data.append(data)
    result_path=to_dir+"/"+file_name
    print("Save:",result_path)
    appended_data = pd.concat(appended_data)
    appended_data.sort_values(by=['ID'], inplace=True)
    appended_data.to_pickle(result_path)
    return appended_data

def predict_CNN(path,to_dir,existing_result):
    appended_pkl = []
    for infile in glob.glob(path):  
     #print("Read:",infile)
        file_name = os.path.basename(infile)
        result_path=to_dir+"/"+file_name.replace("pkl", "ML.pkl")
        if result_path in existing_result :
        # print("found then skip : " , result)
            continue
        else:
            df = pd.read_pickle(infile)
            ready_df =df[[ "reps"]]
            
            X= np.array(df['reps'].to_list())
            X_test = np.reshape(X,(X.shape[0],X.shape[1],1))
            y_probas = learner.predict(X_test)
            threshold = 0.5
            _y = np.where(y_probas > threshold, 1, 0)
            df.drop(columns=['reps'],inplace =True)
            df['class'] = _y
            #print("Save:",result_path)
            df.to_pickle(result_path) 

    print("Complete")



In [5]:
# summarize model.
learner.summary()

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1900, 1)]         0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 1900, 512)         4608      
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 633, 512)          0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 633, 256)          524544    
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 316, 256)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 80896)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 1211)            

## DECockRoach

In [19]:
existing_result = []
for infile in glob.glob("/mnt/vdb/DECockroach/cd100/result_2_CNN_Adam/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

448


In [8]:
# Predict
predict_CNN("/mnt/vdb/DECockroach/cd100/reps/*.pkl","/mnt/vdb/DECockroach/cd100/result_2_CNN_Adam",existing_result)

Complete


In [9]:
# write result
result_df =mergeDF("/mnt/vdb/DECockroach/cd100/result_2_CNN_Adam/*.pkl","/mnt/vdb/DECockroach/cd100/result_2_CNN_Adam","DECockroach.len15.MLResult.plk")
result_df

Save: /mnt/vdb/DECockroach/cd100/result_2_CNN_Adam/DECockroach.len15.MLResult.plk


,ID,length,class
0,0,17,0
1,1,17,0
2,3,18,0
3,4,30,0
4,5,17,0
...,...,...,...
199,894298,17,1
200,894299,19,0
201,894303,17,1
202,894304,17,1


In [10]:
result_df[result_df["class"]== 1]

,ID,length,class
9,10,18,1
10,12,20,1
13,16,19,1
14,17,17,1
16,19,17,1
...,...,...,...
193,894288,17,1
195,894293,28,1
199,894298,17,1
201,894303,17,1


## BAT

In [11]:
existing_result = []
for infile in glob.glob("/mnt/vdb/Bat/cd100/result_2_CNN_Adam/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [13]:
# Predict
predict_CNN("/mnt/vdb/Bat/cd100/reps/*.pkl","/mnt/vdb/Bat/cd100/result_2_CNN_Adam",existing_result)

Complete


In [14]:
# write result
result_df =mergeDF("/mnt/vdb/Bat/cd100/result_2_CNN_Adam/*.pkl","/mnt/vdb/Bat/cd100/result_2_CNN_Adam","Bat.len10.MLResult.plk")
result_df

Save: /mnt/vdb/Bat/cd100/result_2_CNN_Adam/Bat.len10.MLResult.plk


,ID,length,class
0,9,275,1
1,19,47,1
2,32,151,1
3,54,199,1
4,61,152,0
...,...,...,...
74,12568817,51,1
75,12568833,45,1
76,12568849,117,1
77,12568850,174,1


# TransPI

## BAT

In [20]:
existing_result = []
for infile in glob.glob("/mnt/vdb/Bat/transpi/result_2_CNN_RMSpop/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [21]:
# Predict
predict_CNN("/mnt/vdb/Bat/transpi/reps/*.pkl","/mnt/vdb/Bat/transpi/result_2_CNN_RMSpop",existing_result)

Complete


In [22]:
# write result
result_df =mergeDF("/mnt/vdb/Bat/transpi/result_2_CNN_RMSpop/*.pkl","/mnt/vdb/Bat/transpi/result_2_CNN_RMSpop","Bat.len10.MLResult.plk")
result_df

Save: /mnt/vdb/Bat/transpi/result_2_CNN_RMSpop/Bat.len10.MLResult.plk


,ID,length,class
0,SOAP.k25.C303246.p1,101,1
1,SOAP.k25.C303708.p1,102,0
2,SOAP.k25.C304032.p1,103,1
3,SOAP.k25.C304284.p1,104,1
4,SOAP.k25.C304386.p1,104,1
...,...,...,...
1000,Velvet.k61.NODE_783_length_328_cov_36.378048.p1,106,1
1001,Velvet.k61.NODE_8470_length_279_cov_15.172043.p1,105,1
1002,Velvet.k61.NODE_9141_length_765_cov_44.260132.p2,233,1
1003,Velvet.k61.NODE_9415_length_1250_cov_53.216000.p1,367,1


## DECockroach

In [25]:
existing_result = []
for infile in glob.glob("/mnt/vdb/DECockroach/transpi/result_2_CNN_RMSpop/*.pkl"):
    # print(infile)
    existing_result.append(infile)
print(len(existing_result))

0


In [26]:
# Predict
predict_CNN("/mnt/vdb/DECockroach/transpi/reps/*.pkl","/mnt/vdb/DECockroach/transpi/result_2_CNN_RMSpop",existing_result)

Complete


In [27]:
# write result
result_df =mergeDF("/mnt/vdb/DECockroach/transpi/result_2_CNN_RMSpop/*.pkl","/mnt/vdb/DECockroach/transpi/result_2_CNN_RMSpop","DECockroach.len15.MLResult.plk")
result_df

Save: /mnt/vdb/DECockroach/transpi/result_2_CNN_RMSpop/DECockroach.len15.MLResult.plk


,ID,length,class
0,SOAP.k25.C372231.p1,102,1
1,SOAP.k25.C373809.p1,104,1
2,SOAP.k25.C379695.p1,95,1
3,SOAP.k25.C382451.p1,116,1
4,SOAP.k25.C383053.p1,104,1
...,...,...,...
54,Velvet.k37.NODE_9552_length_1035_cov_12.333333.p1,102,1
55,Velvet.k37.NODE_9713_length_2207_cov_17.057997.p1,627,1
56,Velvet.k37.NODE_9748_length_894_cov_40.512302.p1,124,1
57,Velvet.k37.NODE_9800_length_726_cov_45.530304.p1,221,1


In [28]:
result_df[result_df["class"] == 0]

,ID,length,class
9,SOAP.k25.C387573.p1,124,0
19,SOAP.k25.C397059.p1,115,0
23,SOAP.k25.C397729.p1,126,0
31,SOAP.k25.C405333.p1,144,0
32,SOAP.k25.C405897.p1,129,0
...,...,...,...
13,Velvet.k37.NODE_71696_length_822_cov_50.759125.p1,258,0
19,Velvet.k37.NODE_73436_length_1641_cov_16.12309...,514,0
28,Velvet.k37.NODE_7963_length_783_cov_48.134098.p1,157,0
44,Velvet.k37.NODE_8670_length_780_cov_48.342308.p1,254,0
